In [1]:
import json,os

import pandas as pd
import torch
import copy
import numpy as np
import random

from fedavg.server import Server
from fedavg.client import Client,Client_fedprox
from fedavg.models import CNN_Model,weights_init_normal, ReTrainModel,MLP
from utils import get_data

from collections import Counter
from sklearn.preprocessing import MinMaxScaler



In [2]:

data_idx_list=random.sample(range(0,60),10)


In [3]:
mu = 0.05

In [4]:
def min_max_norm(train_datasets, test_dataset, cat_columns, label):
    
    train_data = None
    for key in train_datasets.keys():
        train_datasets[key]['tag'] = key
        train_data = pd.concat([train_data, train_datasets[key]])
    test_dataset['tag'] = key+1
    data = pd.concat([train_data, test_dataset])
    
    min_max = MinMaxScaler()
    con = []

    #查找连续列
    for c in data.columns:
        #TODO 这里写死了使用clinical，后续再修改
        if c not in cat_columns and c not in [label, 'tag']:
            con.append(c)

    data[con] = min_max.fit_transform(data[con])

    # 离散列one-hot
    data = pd.get_dummies(data, columns=cat_columns)
    
    for key in train_datasets.keys():
        c_data = data[data['tag'] == key]
        c_data = c_data.drop(columns=['tag'])
        train_datasets[key] = c_data
    
    test_dataset = data[data['tag'] == key+1]
    test_dataset = test_dataset.drop(columns=['tag'])

    return train_datasets, test_dataset

In [5]:
def test_model(model, test_df):
    """
    model: state_dict
    test_df: test dataset dataframe
    """
    test_dataset = MyImageDataset(test_df,conf["data_column"], conf["label_column"])
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=conf["batch_size"],shuffle=False,drop_last = False)
    
#     test_model = VGG('VGG11').cuda()
    test_model = CNN_Model().cuda()
    test_model.load_state_dict(model)
    
    test_model.eval()
    
    total_loss = 0.0
    correct = 0
    dataset_size = 0

    criterion = torch.nn.CrossEntropyLoss()
    for batch_id, batch in enumerate(test_loader):
        data, target = batch
        dataset_size += data.size()[0]

        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()

        _, output = test_model(data)

        total_loss += criterion(output, target) # sum up batch loss
        pred = output.data.max(1)[1]  # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    acc = 100.0 * (float(correct) / float(dataset_size))
    total_l = total_loss.cpu().detach().numpy() / dataset_size
    return acc, total_l

In [6]:
def model_init(conf, train_datasets, test_dataset, device):
    ###初始化每个节点聚合权值
    client_weight = {}
    if conf["is_init_avg"]:
        for key in train_datasets.keys():
            client_weight[key] = 1 / len(train_datasets)
    print("各节点的聚合权值为：", client_weight)
    
    clients = {}
    
    ##训练目标模型
    if conf['model_name'] == "mlp":
        n_input = test_dataset.shape[1] - 1
        model = MLP(n_input, 512, conf["num_classes"])
    elif conf['model_name'] == 'cnn':
        model = CNN_Model()
    model.apply(weights_init_normal)

    if torch.cuda.is_available():
        model.cuda(device=device)
    
    server = Server(conf, model, test_dataset)
    print("Server初始化完成!")
    
    for key in train_datasets.keys():
        clients[key] = Client_fedprox(conf, copy.deepcopy(server.global_model), train_datasets[key])#new_add
    print("参与方初始化完成！")

    #保存模型
    if not os.path.isdir(conf["model_dir"]):
        os.mkdir(conf["model_dir"])
    
    return clients, server, client_weight

In [7]:
def train_and_eval(clients, server, client_weight):
    #联邦训练
    loss_list = []
    acc_list = []
    roc_list = []
    max_auc = 0
    max_acc = 0
    maxe = 0
    maxe2 = 0
    for e in range(conf["global_epochs"]):
        global_weight_collector = list((server.global_model).parameters())#new_add
        clients_models = {}
        for key in clients.keys():
#             print('training client {}...'.format(key))
            model_k = clients[key].local_train(server.global_model,global_weight_collector,mu)#new_add
            clients_models[key] = copy.deepcopy(model_k)

    #         acc, loss = test_model(clients_models[key], test_dataset)
    #         print("client %d,Epoch %d, global_acc: %f, global_loss: %f\n" % (key, e, acc, loss))


        #联邦聚合
        server.model_aggregate(clients_models, client_weight)

        #测试全局模型
        acc, loss, auc_roc, f1 = server.model_eval()
        loss_list.append(loss)
        acc_list.append(acc)
        roc_list.append(auc_roc)
#         auc_roc, loss = server.model_eval()
    #     print("Epoch %d, global_acc: %f, global_loss: %f, auc_roc: %f, f1: %f\n" % (e, acc, loss, auc_roc, f1))
        print("Epoch %d, global_loss: %f, auc_roc: %f" % (e, loss, auc_roc))

        #保存最好的模型
        if auc_roc > max_auc:
            torch.save(server.global_model.state_dict(), 
                       os.path.join(conf["model_dir"], "roc_model_best_{}.pth".format(e)))
            
            for idx,_ in enumerate(clients_models):
                torch.save(clients_models[idx], 
                           os.path.join(conf["model_dir"], "roc_model_best_l{}_{}.pth".format(idx,e)))
#                 print(clients_models[idx])
#                 print("roc")
                exit()
                idx += 1
            
#             print("model save done !")
            max_auc = auc_roc
            maxe = e
        if acc > max_acc:
            torch.save(server.global_model.state_dict(), 
                       os.path.join(conf["model_dir"], "acc_model_best.pth"))
#             print("model save done !")
            
            for idx, _ in enumerate(clients_models):
                torch.save(clients_models[idx], 
                           os.path.join(conf["model_dir"], "acc_model_best_l{}.pth".format(idx)))
#                 print(clients_models[idx])
#                 print("acc")
#                 exit()
            max_acc = acc
            maxe2 = e

    print('max auc = {0}, epoch = {1}'.format(max_auc, maxe))
    print('max acc = {0}, epoch = {1}'.format(max_acc, maxe2))
#     loss_list.to_csv(os.path.join(conf["model_dir"], "loss_list.csv"),index=False)
#     acc_list.to_csv(os.path.join(conf["model_dir"], "acc_list.csv"),index=False)
#     roc_list.to_csv(os.path.join(conf["model_dir"], "roc_list.csv"),index=False)
    save = [loss_list,acc_list,roc_list]
    name = ["loss","acc","roc"]
    record = pd.DataFrame(data=save)
    record.to_csv(os.path.join(conf["model_dir"], "record_list.csv"),index=False)
    return max_auc

In [8]:
def base_train(conf, dataset_name, b, clients_num, path, label_name, train_gpu):
    # 构造文件读取路径
    train_files_path_list = [path + "beta{}/".format(b) + dataset_name + "_node_{}.csv".format(i) for i in range(clients_num)]
    print("划分数据目录如下:\n" + str(train_files_path_list))
    
    # 读取文件
    train_datasets = {}
    for i in range(len(train_files_path_list)):
        train_datasets[i] = pd.read_csv(train_files_path_list[i])
        print(train_datasets[i][label_name].value_counts())
    test_dataset = pd.read_csv(path + '{}_test.csv'.format(dataset_name))
    print("测试数据格式如下: " + str(test_dataset.shape))
    
    train_datasets, test_dataset = min_max_norm(
        train_datasets, test_dataset, 
        conf['discrete_columns'][dataset_name], 
        conf['label_column']) 
    
    clients, server, client_weight = model_init(conf, train_datasets, test_dataset, train_gpu)
    
    max_score = train_and_eval(clients, server, client_weight)
    
    return max_score

In [9]:
def base_train_60(conf, dataset_name, b, clients_num, path, label_name, train_gpu):
    
    # 构造文件读取路径
    data_idx_list=random.sample(range(0,60),clients_num)
    train_files_path_list = [path + "beta{}/".format(b) + dataset_name + "_node_{}.csv".format(data_idx_list[i]) for i in range(clients_num)]
    print("划分数据目录如下:\n" + str(train_files_path_list))
    
    # 读取文件
    train_datasets = {}
    for i in range(len(train_files_path_list)):
        train_datasets[i] = pd.read_csv(train_files_path_list[i])
        print(train_datasets[i][label_name].value_counts())
    test_dataset = pd.read_csv(path + '{}_test.csv'.format(dataset_name))
    print("测试数据格式如下: " + str(test_dataset.shape))
    
    train_datasets, test_dataset = min_max_norm(
        train_datasets, test_dataset, 
        conf['discrete_columns'][dataset_name], 
        conf['label_column']) 
    
    clients, server, client_weight = model_init(conf, train_datasets, test_dataset, train_gpu)
    
    max_score = train_and_eval(clients, server, client_weight)
    
    return max_score


In [10]:
def get_norm_test(conf, dataset_name, b, clients_num, path, label_name, train_gpu):
    # 构造文件读取路径
    train_files_path_list = [path + "beta{}/".format(b) + dataset_name + "_node_{}.csv".format(i) for i in range(clients_num)]
    print("划分数据目录如下:\n" + str(train_files_path_list))
    
    # 读取文件
    train_datasets = {}
    for i in range(len(train_files_path_list)):
        train_datasets[i] = pd.read_csv(train_files_path_list[i])
        print(train_datasets[i][label_name].value_counts())
    test_dataset = pd.read_csv(path + '{}_test.csv'.format(dataset_name))
    print("测试数据格式如下: " + str(test_dataset.shape))
    
    train_datasets, test_dataset = min_max_norm(
        train_datasets, test_dataset, 
        conf['discrete_columns'][dataset_name], 
        conf['label_column']) 
    test_dataset.to_csv(path + "beta{}/".format(b) + dataset_name + "_norm_test.csv",index=False)
    
    return 

In [11]:
def get_random_data(syn_data, aug_numbers,label,ratio):
    """
    从合成数据中采样增强
    """
  
    aug_data = None   
    for i in range(len(aug_numbers)):
        
        aug_i = syn_data[syn_data[label] == i]
        if aug_i.shape[0] >= aug_numbers[i]:
            aug_data = pd.concat([aug_data, aug_i.sample(int(ratio * aug_numbers[i]))])
        else:
            print('label {} has no enough synthetic data'.format(i))
    
    return aug_data
        
def random_aug(train_datasets, path, dataset_name, label, label_num,  aug_type='same_number'):
    """
    随机增强
    """ 
    labels_dis = []
    
    for key in train_datasets.keys():
        label_dis = []
        for i in range(label_num):
            label_i = len(train_datasets[key][train_datasets[key][label] == i])
            label_dis.append(label_i)
        labels_dis.append(label_dis)
    labels_dis = np.array(labels_dis)
    print(labels_dis)
    total_dis = np.sum(labels_dis, axis=0)
    print(total_dis)
    aug_numbers = total_dis - labels_dis
    print("len:",train_datasets.keys())
    ratio = 1/len(train_datasets.keys())
    if aug_type == 'same_number':
        
        for key in train_datasets.keys():
#             syn_data = pd.read_csv('./data/clinical/syn_data/clinical_syn_{}.csv'.format(key))
            syn_data = pd.read_csv('{0}/{1}_syn.csv'.format(path, dataset_name, key))
            aug_data = get_random_data(syn_data, aug_numbers[key],label,ratio)
            train_datasets[key] = pd.concat([train_datasets[key], aug_data])
            print(train_datasets[key].shape)
    
    return train_datasets

In [12]:
def random_aug_from_one(train_datasets, path, dataset_name, label, label_num,  aug_type='same_number'):
    """
    随机增强
    """ 
    labels_dis = []
    
    for key in train_datasets.keys():
        label_dis = []
        for i in range(label_num):
            label_i = len(train_datasets[key][train_datasets[key][label] == i])
            label_dis.append(label_i)
        labels_dis.append(label_dis)
    labels_dis = np.array(labels_dis)
    print(labels_dis)
    total_dis = np.sum(labels_dis, axis=0)
    print(total_dis)
    aug_numbers = total_dis - labels_dis

    if aug_type == 'same_number':
        
        for key in train_datasets.keys():
#             syn_data = pd.read_csv('./data/clinical/syn_data/clinical_syn_{}.csv'.format(key))
            syn_data = pd.read_csv('{0}/{1}_syn.csv'.format(path, dataset_name, key))
            aug_data = get_random_data(syn_data, aug_numbers[key],label)
            train_datasets[key] = pd.concat([train_datasets[key], aug_data])
            print(train_datasets[key].shape)
    
    return train_datasets

In [13]:
def augment_train(conf, dataset_name, b, clients_num, path, label_name, label_num, augment_path, train_gpu):
    # 构造文件读取路径
    train_files_path_list = [path + "beta{}/".format(b) + dataset_name + "_node_{}.csv".format(i) for i in range(clients_num)]
    print("划分数据目录如下:\n" + str(train_files_path_list))
    
    # 读取文件
    train_datasets = {}
    for i in range(len(train_files_path_list)):
        train_datasets[i] = pd.read_csv(train_files_path_list[i])
        print(train_datasets[i][label_name].value_counts())
    test_dataset = pd.read_csv(path + '{}_test.csv'.format(dataset_name))
    print("测试数据格式如下: " + str(test_dataset.shape))
    
    train_datasets = random_aug(train_datasets, augment_path, dataset_name, label_name, label_num)
    
    train_datasets, test_dataset = min_max_norm(
        train_datasets, test_dataset, 
        conf['discrete_columns'][dataset_name], 
        conf['label_column']) 
    
    clients, server, client_weight = model_init(conf, train_datasets, test_dataset, train_gpu)
    
    max_score = train_and_eval(clients, server, client_weight)
    
    return max_score

In [14]:
def get_norm_augment_test(conf, dataset_name, b, clients_num, path, label_name, label_num, augment_path, train_gpu):
    # 构造文件读取路径
    train_files_path_list = [path + "beta{}/".format(b) + dataset_name + "_node_{}.csv".format(i) for i in range(clients_num)]
    print("划分数据目录如下:\n" + str(train_files_path_list))
    
    # 读取文件
    train_datasets = {}
    for i in range(len(train_files_path_list)):
        train_datasets[i] = pd.read_csv(train_files_path_list[i])
        print(train_datasets[i][label_name].value_counts())
    test_dataset = pd.read_csv(path + '{}_test.csv'.format(dataset_name))
    print("测试数据格式如下: " + str(test_dataset.shape))
    
    train_datasets = random_aug(train_datasets, augment_path, dataset_name, label_name, label_num)
    
    train_datasets, test_dataset = min_max_norm(
        train_datasets, test_dataset, 
        conf['discrete_columns'][dataset_name], 
        conf['label_column']) 
    
    test_dataset.to_csv(path + "beta{}/".format(b) + dataset_name + "_norm_augment_test.csv",index=False)
    return

In [15]:
def augment_train_in_60(conf, dataset_name, b, clients_num, path, label_name, label_num, augment_path, train_gpu):
    # 构造文件读取路径
    data_idx_list=random.sample(range(0,60),clients_num)
    train_files_path_list = [path + "beta{}/".format(b) + dataset_name + "_node_{}.csv".format(data_idx_list[i]) for i in range(clients_num)]
    print("划分数据目录如下:\n" + str(train_files_path_list))
    
    # 读取文件
    train_datasets = {}
    for i in range(len(train_files_path_list)):
        train_datasets[i] = pd.read_csv(train_files_path_list[i])
        print(train_datasets[i][label_name].value_counts())
    test_dataset = pd.read_csv(path + '{}_test.csv'.format(dataset_name))
    print("测试数据格式如下: " + str(test_dataset.shape))
    
    train_datasets = random_aug(train_datasets, augment_path, dataset_name, label_name, label_num)
    
    train_datasets, test_dataset = min_max_norm(
        train_datasets, test_dataset, 
        conf['discrete_columns'][dataset_name], 
        conf['label_column']) 
    
    clients, server, client_weight = model_init(conf, train_datasets, test_dataset, train_gpu)
    
    max_score = train_and_eval(clients, server, client_weight)
    
    return max_score

In [16]:
def gan_augment_train(conf, dataset_name, b, clients_num, path, label_name, label_num, augment_path, train_gpu):
    # 构造文件读取路径
    train_files_path_list = [path + "b={}/".format(b) + label_name + "_{}.csv".format(i) for i in range(clients_num)]
    print("划分数据目录如下:\n" + str(train_files_path_list))
    
    # 读取文件
    train_datasets = {}
    for i in range(len(train_files_path_list)):
        train_datasets[i] = pd.read_csv(train_files_path_list[i])
        print(train_datasets[i][label_name].value_counts())
    test_dataset = pd.read_csv(path + '{}_test.csv'.format(dataset_name))
    print("测试数据格式如下: " + str(test_dataset.shape))
    
    train_datasets = random_aug_from_one(train_datasets, augment_path, dataset_name, label_name, label_num)
    
    train_datasets, test_dataset = min_max_norm(
        train_datasets, test_dataset, 
        conf['discrete_columns'][dataset_name], 
        conf['label_column']) 
    
    clients, server, client_weight = model_init(conf, train_datasets, test_dataset, train_gpu)
    
    max_score = train_and_eval(clients, server, client_weight)
    
    return max_score

# clinical 

In [17]:

##配置文件
conf = {

	#数据类型，tabular, image
	"data_type" : "tabular",

	#选择模型mlp,simple-cnn,vgg
	"model_name" : "mlp",

	#处理方法:fed_ccvr
	"no-iid": "",

	#全局epoch
	"global_epochs" : 100,

	#本地epoch
	"local_epochs" : 3,

	#狄利克雷参数
	"beta" : 0.05,

	"batch_size" : 64,

	"weight_decay":1e-5,

    #学习速率
	"lr" : 0.001,

	"momentum" : 0.9,

	#分类
	"num_classes": 2,

	#节点数
	"num_parties":5,

    #模型聚合权值
	"is_init_avg": True,

    #本地验证集划分比例
	"split_ratio": 0.2,

    #标签列名
	"label_column": "label",

	#数据列名
	"data_column": "file",

    #测试数据
    "test_dataset": "../实验1/dp-fedavg-gan/clinical/clinical_test.csv",

    #训练数据
    "train_dataset": "../实验1/dp-fedavg-gan/clinical/clinical_train.csv",

    #模型保存目录
	"model_dir":"./save_model/clinical/",

    #模型文件名
	"model_file":"model.pth",

	"retrain":{
		"epoch": 10,
		"lr": 0.001,
		"num_vr":2000
	},
    
    "discrete_columns": {
        "adult":[
            'workclass',
            'education',
            'marital_status',
            'occupation',
            'relationship',
            'race',
            'gender',
            'native_country'
        ],
        "intrusion":['protocol_type', 'service', 'flag'],
        "credit":[],
        "covtype":
            ['Wilderness_Area_0', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3', 'Soil_Type0', 'Soil_Type1',
             'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9',
             'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 
             'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 
             'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 
             'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 
             'Soil_Type38', 'Soil_Type39'],
        "clinical":["anaemia","diabetes","high_blood_pressure","sex","smoking"]
    }
}

# covtype

In [28]:

##配置文件
conf = {

	#数据类型，tabular, image
	"data_type" : "tabular",

	#选择模型mlp,simple-cnn,vgg
	"model_name" : "mlp",

	#处理方法:fed_ccvr
	"no-iid": "",

	#全局epoch
	"global_epochs" : 100,

	#本地epoch
	"local_epochs" : 3,

	#狄利克雷参数
	"beta" : 0.05,

	"batch_size" : 65,

	"weight_decay":1e-5,

    #学习速率
	"lr" : 0.001,

	"momentum" : 0.9,

	#分类
	"num_classes": 7,

	#节点数
	"num_parties":5,

    #模型聚合权值
	"is_init_avg": True,

    #本地验证集划分比例
	"split_ratio": 0.2,

    #标签列名
	"label_column": "Cover_Type",

	#数据列名
	"data_column": "covtype",

    #测试数据
    "test_dataset": "../实验1/dp-fedavg-gan/covtype/covtype_test.csv",

    #训练数据
    "train_dataset": "../实验1/dp-fedavg-gan/covtype/covtype_train.csv",

    #模型保存目录
	"model_dir":"./save_model/covtype/",

    #模型文件名
	"model_file":"model.pth",

	"retrain":{
		"epoch": 10,
		"lr": 0.001,
		"num_vr":2000
	},
    
    "discrete_columns": {
        "adult":[
            'workclass',
            'education',
            'marital_status',
            'occupation',
            'relationship',
            'race',
            'gender',
            'native_country'
        ],
        "intrusion":['protocol_type', 'service', 'flag'],
        "credit":[],
        "covtype":
            ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1',
             'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9',
             'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 
             'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 
             'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 
             'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 
             'Soil_Type38', 'Soil_Type39', 'Soil_Type40'],
        "clinical":["anaemia","diabetes","high_blood_pressure","sex","smoking"]
    }
}

# tb

In [33]:

##配置文件
conf = {

	#数据类型，tabular, image
	"data_type" : "tabular",

	#选择模型mlp,simple-cnn,vgg
	"model_name" : "mlp",

	#处理方法:fed_ccvr
	"no-iid": "",

	#全局epoch
	"global_epochs" : 100,

	#本地epoch
	"local_epochs" : 3,

	#狄利克雷参数
	"beta" : 0.05,

	"batch_size" : 65,

	"weight_decay":1e-5,

    #学习速率
	"lr" : 0.001,

	"momentum" : 0.9,

	#分类
	"num_classes": 2,

	#节点数
	"num_parties":5,

    #模型聚合权值
	"is_init_avg": True,

    #本地验证集划分比例
	"split_ratio": 0.2,

    #标签列名
	"label_column": "Condition",

	#数据列名
	"data_column": "tb",

    #测试数据
    "test_dataset": "../实验1/dp-fedavg-gan/tb/tb_test.csv",

    #训练数据
    "train_dataset": "../实验1/dp-fedavg-gan/tb/tb_train.csv",

    #模型保存目录
	"model_dir":"./save_model/tb/",

    #模型文件名
	"model_file":"model.pth",

	"retrain":{
		"epoch": 10,
		"lr": 0.001,
		"num_vr":2000
	},
    
    "discrete_columns": {
        "adult": [
            'workclass',
            'education',
            'marital_status',
            'occupation',
            'relationship',
            'race',
            'gender',
            'native_country'
        ],
        "intrusion": ['protocol_type', 'service', 'flag'],
        "credit": [],
        "tb": [],
        "covtype":
            ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1',
             'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9',
             'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 
             'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 
             'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 
             'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 
             'Soil_Type38', 'Soil_Type39', 'Soil_Type40'],
        "clinical": ["anaemia","diabetes","high_blood_pressure","sex","smoking"]
    }
}

# intrusion

In [25]:

##配置文件
conf = {

	#数据类型，tabular, image
	"data_type" : "tabular",

	#选择模型mlp,simple-cnn,vgg
	"model_name" : "mlp",

	#处理方法:fed_ccvr
	"no-iid": "",

	#全局epoch
	"global_epochs" : 100,

	#本地epoch
	"local_epochs" : 5,

	#狄利克雷参数
	"beta" : 0.05,

	"batch_size" : 64,

	"weight_decay":1e-5,

    #学习速率
	"lr" : 0.001,

	"momentum" : 0.9,

	#分类
	"num_classes": 10,

	#节点数
	"num_parties":5,

    #模型聚合权值
	"is_init_avg": True,

    #本地验证集划分比例
	"split_ratio": 0.2,

    #标签列名
	"label_column": "label",

	#数据列名
	"data_column": "intrusion",

    #测试数据
#     "test_dataset": "../实验1/dp-fedavg-gan/covtype/covtype_test.csv",

    #训练数据
#     "train_dataset": "../实验1/dp-fedavg-gan/covtype/covtype_train.csv",

    #模型保存目录
	"model_dir":"./save_model/intrusion/",

    #模型文件名
	"model_file":"model.pth",

	"retrain":{
		"epoch": 10,
		"lr": 0.001,
		"num_vr":2000
	},
    
    "discrete_columns": {
        "adult":[
            'workclass',
            'education',
            'marital_status',
            'occupation',
            'relationship',
            'race',
            'gender',
            'native_country'
        ],
        "intrusion":['protocol_type', 'service', 'flag'],
        "credit":[],
        "covtype":
            ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1',
             'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9',
             'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 
             'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 
             'Soil_Type24', 'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 
             'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 
             'Soil_Type38', 'Soil_Type39', 'Soil_Type40'],
        "clinical":["anaemia","diabetes","high_blood_pressure","sex","smoking"]
    }
}

In [33]:
score = []
for i in range(5):
    score.append(augment_train(conf, "covtype", 0.01, 5, 
                  "./data/covtype/tmp/",
                  "Cover_Type", 10, 
                  "./data/syn_data/covtype",0))

划分数据目录如下:
['./data/covtype/tmp/beta0.01/covtype_node_0.csv', './data/covtype/tmp/beta0.01/covtype_node_1.csv', './data/covtype/tmp/beta0.01/covtype_node_2.csv', './data/covtype/tmp/beta0.01/covtype_node_3.csv', './data/covtype/tmp/beta0.01/covtype_node_4.csv']
0    12919
Name: Cover_Type, dtype: int64
5    1215
3     111
2      13
Name: Cover_Type, dtype: int64
1    19831
Name: Cover_Type, dtype: int64
3    80
Name: Cover_Type, dtype: int64
2    2489
0    1909
6    1435
4     664
3       1
Name: Cover_Type, dtype: int64
测试数据格式如下: (174308, 55)
[[12919     0     0     0     0     0     0     0     0     0]
 [    0     0    13   111     0  1215     0     0     0     0]
 [    0 19831     0     0     0     0     0     0     0     0]
 [    0     0     0    80     0     0     0     0     0     0]
 [ 1909     0  2489     1   664     0  1435     0     0     0]]
[14828 19831  2502   192   664  1215  1435     0     0     0]
len: dict_keys([0, 1, 2, 3, 4])
(18466, 55)
(9202, 55)
(23996, 55)
(8195,

K:\Workspace\fedprox_pack\fedprox\fedavg\models.py:109: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight, 0.0, 0.01)
K:\Workspace\fedprox_pack\fedprox\fedavg\models.py:110: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  torch.nn.init.constant(m.bias, 0.0)
K:\Workspace\fedprox_pack\fedprox\fedavg\models.py:106: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight, 1.0, 0.02)
K:\Workspace\fedprox_pack\fedprox\fedavg\models.py:107: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  torch.nn.init.constant(m.bias, 0.0)


Epoch 0 done. train_loss =1.018852710723877, eval_loss = 0.01268653287074683, eval_acc=85.82800823134409
Epoch 1 done. train_loss =0.9256502389907837, eval_loss = 0.007683815011530905, eval_acc=89.01223870897866
Epoch 2 done. train_loss =1.0629589557647705, eval_loss = 0.008499067007598001, eval_acc=89.73248131701506
Epoch 0 done. train_loss =1.4673649072647095, eval_loss = 0.01764885401420038, eval_acc=55.68354705498805
Epoch 1 done. train_loss =1.4569776058197021, eval_loss = 0.015851170673134067, eval_acc=57.878722016952835
Epoch 2 done. train_loss =1.481877326965332, eval_loss = 0.01676917537298702, eval_acc=56.20517278852424
Epoch 0 done. train_loss =0.7693158388137817, eval_loss = 0.0043630153263662435, eval_acc=94.29071511918653
Epoch 1 done. train_loss =1.2811267375946045, eval_loss = 0.0039595600604454585, eval_acc=93.84897482913819
Epoch 2 done. train_loss =1.5760127305984497, eval_loss = 0.0039145944515532865, eval_acc=93.7031171861977
Epoch 0 done. train_loss =1.97841119766

Epoch 1 done. train_loss =0.665909469127655, eval_loss = 0.008756550862569159, eval_acc=75.31882970742686
Epoch 2 done. train_loss =0.9014886617660522, eval_loss = 0.008526485983983789, eval_acc=75.97899474868717
roc_auc = 0, f1_score=0, acc=48.76253528237373
Epoch 4, global_loss: 0.016951, auc_roc: 0.000000
Epoch 0 done. train_loss =1.0974223613739014, eval_loss = 0.007396452168427786, eval_acc=89.84078847611828
Epoch 1 done. train_loss =0.3590971827507019, eval_loss = 0.005771075479777446, eval_acc=90.64226145348208
Epoch 2 done. train_loss =0.3533095717430115, eval_loss = 0.00570979490706738, eval_acc=90.48521607278242
Epoch 0 done. train_loss =0.8764412999153137, eval_loss = 0.014987594846589906, eval_acc=59.04151271462725
Epoch 1 done. train_loss =1.3563520908355713, eval_loss = 0.014895528069114561, eval_acc=60.302108237339716
Epoch 2 done. train_loss =1.1016075611114502, eval_loss = 0.01471863257265744, eval_acc=59.98695935666159
Epoch 0 done. train_loss =1.497625470161438, eval

Epoch 2 done. train_loss =1.6690617799758911, eval_loss = 0.01776725391995405, eval_acc=53.23978035387431
Epoch 0 done. train_loss =1.184146761894226, eval_loss = 0.00844405475453336, eval_acc=76.81170292573142
Epoch 1 done. train_loss =0.7486964464187622, eval_loss = 0.008373562697620206, eval_acc=76.25656414103526
Epoch 2 done. train_loss =0.689117431640625, eval_loss = 0.008472024240801412, eval_acc=76.54163540885222
roc_auc = 0, f1_score=0, acc=56.863138811758496
Epoch 9, global_loss: 0.015125, auc_roc: 0.000000
Epoch 0 done. train_loss =0.3093827962875366, eval_loss = 0.00567812833369753, eval_acc=90.60976930575112
Epoch 1 done. train_loss =0.9058709144592285, eval_loss = 0.005766293164078365, eval_acc=89.55377450449474
Epoch 2 done. train_loss =0.3589310646057129, eval_loss = 0.006191504036477724, eval_acc=90.60976930575112
Epoch 0 done. train_loss =0.992081880569458, eval_loss = 0.014594665275296811, eval_acc=60.38904585959573
Epoch 1 done. train_loss =0.9982693195343018, eval_l

...

Epoch 0 done. train_loss =2.1859865188598633, eval_loss = 0.008084198593288459, eval_acc=77.02175543885971
Epoch 1 done. train_loss =0.5102516412734985, eval_loss = 0.007953582557626711, eval_acc=77.14928732183046
Epoch 2 done. train_loss =1.2586283683776855, eval_loss = 0.007845624049444978, eval_acc=77.69692423105776
roc_auc = 0, f1_score=0, acc=59.70179222984602
Epoch 77, global_loss: 0.015179, auc_roc: 0.000000
Epoch 0 done. train_loss =0.16182532906532288, eval_loss = 0.0052106066720012875, eval_acc=90.86970648759883
Epoch 1 done. train_loss =1.0764105319976807, eval_loss = 0.005322914449930785, eval_acc=90.86429112964368
Epoch 2 done. train_loss =0.20447304844856262, eval_loss = 0.005143222914193442, eval_acc=90.9563522148814
Epoch 0 done. train_loss =0.9841369390487671, eval_loss = 0.014353723324943796, eval_acc=61.43229732666812
Epoch 1 done. train_loss =0.8839467167854309, eval_loss = 0.01430554422081307, eval_acc=61.269289284938054
Epoch 2 done. train_loss =1.071373701095581,

Epoch 1 done. train_loss =1.6212347745895386, eval_loss = 0.017417752240910044, eval_acc=54.66748017083588
Epoch 2 done. train_loss =0.92413729429245, eval_loss = 0.017474994577962935, eval_acc=54.75289810860281
Epoch 0 done. train_loss =1.0007574558258057, eval_loss = 0.007891479972959548, eval_acc=77.72693173293324
Epoch 1 done. train_loss =0.8494739532470703, eval_loss = 0.00773838269290223, eval_acc=77.82445611402851
Epoch 2 done. train_loss =3.4824106693267822, eval_loss = 0.008163724013583963, eval_acc=76.9317329332333
roc_auc = 0, f1_score=0, acc=58.20731119627326
Epoch 82, global_loss: 0.015182, auc_roc: 0.000000
Epoch 0 done. train_loss =0.33537375926971436, eval_loss = 0.005222564320464596, eval_acc=90.86429112964368
Epoch 1 done. train_loss =0.8165889978408813, eval_loss = 0.00520731833933183, eval_acc=90.8317989819127
Epoch 2 done. train_loss =0.6405971050262451, eval_loss = 0.005215648864726262, eval_acc=90.82096826600238
Epoch 0 done. train_loss =0.7846252918243408, eval_

Epoch 2 done. train_loss =0.6276638507843018, eval_loss = 0.003046196010594051, eval_acc=94.56159359893316
Epoch 0 done. train_loss =0.7334318161010742, eval_loss = 0.01739674184728207, eval_acc=54.996949359365466
Epoch 1 done. train_loss =1.956586241722107, eval_loss = 0.017546190467820508, eval_acc=54.008541793776686
Epoch 2 done. train_loss =1.1713664531707764, eval_loss = 0.017521841574035236, eval_acc=55.143380109823056
Epoch 0 done. train_loss =0.7916024923324585, eval_loss = 0.008204248518489487, eval_acc=76.43660915228807
Epoch 1 done. train_loss =0.8262424468994141, eval_loss = 0.008091460540611228, eval_acc=77.09677419354838
Epoch 2 done. train_loss =1.833392858505249, eval_loss = 0.008436054478707567, eval_acc=75.91147786946738
roc_auc = 0, f1_score=0, acc=55.24244440874774
Epoch 87, global_loss: 0.016085, auc_roc: 0.000000
Epoch 0 done. train_loss =0.14364264905452728, eval_loss = 0.005238287087110686, eval_acc=90.8047221921369
Epoch 1 done. train_loss =0.37452712655067444,

Epoch 0 done. train_loss =0.1407240331172943, eval_loss = 0.0029739167495933566, eval_acc=94.58243040506751
Epoch 1 done. train_loss =0.22304940223693848, eval_loss = 0.002965121134894553, eval_acc=94.57409568261377
Epoch 2 done. train_loss =1.1436433792114258, eval_loss = 0.0031258821050253165, eval_acc=94.30738456409402
Epoch 0 done. train_loss =0.7011770009994507, eval_loss = 0.017409781176117297, eval_acc=54.9725442342892
Epoch 1 done. train_loss =0.7969892024993896, eval_loss = 0.017418716737888005, eval_acc=55.22879804758999
Epoch 2 done. train_loss =1.591989278793335, eval_loss = 0.017459644552699816, eval_acc=54.83831604636974
Epoch 0 done. train_loss =0.5397891402244568, eval_loss = 0.0077976813463009305, eval_acc=77.86946736684172
Epoch 1 done. train_loss =0.8859838843345642, eval_loss = 0.00780112745166272, eval_acc=77.81695423855965
Epoch 2 done. train_loss =0.48191094398498535, eval_loss = 0.008041095662099118, eval_acc=76.8867216804201
roc_auc = 0, f1_score=0, acc=55.7616

Epoch 0 done. train_loss =1.1098190546035767, eval_loss = 0.014464055214516885, eval_acc=60.791132362529886
Epoch 1 done. train_loss =1.0822207927703857, eval_loss = 0.014324228858408838, eval_acc=61.53010215170616
Epoch 2 done. train_loss =1.0970582962036133, eval_loss = 0.0142467343737265, eval_acc=60.82373397087589
Epoch 0 done. train_loss =0.48221614956855774, eval_loss = 0.0029520911362831306, eval_acc=94.6116019336556
Epoch 1 done. train_loss =0.15758436918258667, eval_loss = 0.0030008555452194505, eval_acc=94.56576096016003
Epoch 2 done. train_loss =0.4090735912322998, eval_loss = 0.0029600095423008327, eval_acc=94.58243040506751
Epoch 0 done. train_loss =2.2988786697387695, eval_loss = 0.017473538522679608, eval_acc=54.75289810860281
Epoch 1 done. train_loss =0.908992350101471, eval_loss = 0.01744236553348078, eval_acc=54.26479560707749
Epoch 2 done. train_loss =0.6677942872047424, eval_loss = 0.01747464266690341, eval_acc=54.77730323367908
Epoch 0 done. train_loss =0.771738946

In [34]:
print(score)

[0, 0, 0, 0, 0]


In [30]:
score_base = []
for i in range(5):
    score_base.append(base_train(conf, "covtype", 0.01, 5, "./data/covtype/tmp/", "Cover_Type",0))

划分数据目录如下:
['./data/covtype/tmp/beta0.01/covtype_node_0.csv', './data/covtype/tmp/beta0.01/covtype_node_1.csv', './data/covtype/tmp/beta0.01/covtype_node_2.csv', './data/covtype/tmp/beta0.01/covtype_node_3.csv', './data/covtype/tmp/beta0.01/covtype_node_4.csv']
0    12919
Name: Cover_Type, dtype: int64
5    1215
3     111
2      13
Name: Cover_Type, dtype: int64
1    19831
Name: Cover_Type, dtype: int64
3    80
Name: Cover_Type, dtype: int64
2    2489
0    1909
6    1435
4     664
3       1
Name: Cover_Type, dtype: int64
测试数据格式如下: (174308, 55)
各节点的聚合权值为： {0: 0.2, 1: 0.2, 2: 0.2, 3: 0.2, 4: 0.2}
Server初始化完成!
参与方初始化完成！


K:\Workspace\fedprox_pack\fedprox\fedavg\models.py:109: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight, 0.0, 0.01)
K:\Workspace\fedprox_pack\fedprox\fedavg\models.py:110: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  torch.nn.init.constant(m.bias, 0.0)
K:\Workspace\fedprox_pack\fedprox\fedavg\models.py:106: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(m.weight, 1.0, 0.02)
K:\Workspace\fedprox_pack\fedprox\fedavg\models.py:107: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  torch.nn.init.constant(m.bias, 0.0)


Epoch 0 done. train_loss =0.8702684640884399, eval_loss = 0.003802078852980893, eval_acc=100.0
Epoch 1 done. train_loss =0.6721664667129517, eval_loss = 0.0008422907074775403, eval_acc=100.0
Epoch 2 done. train_loss =0.49294623732566833, eval_loss = 0.0005567197654258172, eval_acc=100.0
Epoch 0 done. train_loss =1.944691777229309, eval_loss = 0.029121877543511365, eval_acc=90.739357729649
Epoch 1 done. train_loss =1.6201198101043701, eval_loss = 0.022981957057414435, eval_acc=94.54817027632562
Epoch 2 done. train_loss =1.3670517206192017, eval_loss = 0.01537430829481904, eval_acc=95.3696788648245
Epoch 0 done. train_loss =0.7602097988128662, eval_loss = 0.001436155326042658, eval_acc=100.0
Epoch 1 done. train_loss =0.4946327209472656, eval_loss = 0.0007638917328291741, eval_acc=100.0
Epoch 2 done. train_loss =0.7664425373077393, eval_loss = 0.0008313984669172772, eval_acc=100.0
Epoch 0 done. train_loss =1.9534052610397339, eval_loss = 0.0485593855381012, eval_acc=100.0
Epoch 1 done. tr

Epoch 0 done. train_loss =2.123871088027954, eval_loss = 0.028305999550737492, eval_acc=6.348020911127707
Epoch 1 done. train_loss =1.2346837520599365, eval_loss = 0.015392497193020969, eval_acc=90.29126213592234
Epoch 2 done. train_loss =0.8672137260437012, eval_loss = 0.010724071961360871, eval_acc=94.0253920836445
Epoch 0 done. train_loss =0.2364930510520935, eval_loss = 0.0005348460059093579, eval_acc=100.0
Epoch 1 done. train_loss =0.13453669846057892, eval_loss = 0.0002920217986763543, eval_acc=100.0
Epoch 2 done. train_loss =0.21636897325515747, eval_loss = 0.00024005542105141474, eval_acc=100.0
Epoch 0 done. train_loss =3.62758731842041, eval_loss = 0.06548073291778564, eval_acc=0.0
Epoch 1 done. train_loss =3.843900680541992, eval_loss = 0.06935064792633057, eval_acc=0.0
Epoch 2 done. train_loss =3.55975604057312, eval_loss = 0.0713255226612091, eval_acc=0.0
Epoch 0 done. train_loss =0.6920322775840759, eval_loss = 0.003076855679151277, eval_acc=94.41366574330563
Epoch 1 done.

Epoch 0 done. train_loss =0.11287714540958405, eval_loss = 0.0003373949786585318, eval_acc=100.0
Epoch 1 done. train_loss =0.08622901141643524, eval_loss = 0.00020759986091457188, eval_acc=100.0
Epoch 2 done. train_loss =0.09000708162784576, eval_loss = 0.00017351681234784762, eval_acc=100.0
Epoch 0 done. train_loss =4.708545207977295, eval_loss = 0.07633543610572815, eval_acc=0.0
Epoch 1 done. train_loss =4.5952677726745605, eval_loss = 0.07332929372787475, eval_acc=0.0
Epoch 2 done. train_loss =4.228109359741211, eval_loss = 0.0738204300403595, eval_acc=0.0
Epoch 0 done. train_loss =0.23216000199317932, eval_loss = 0.002520807488730006, eval_acc=94.65989535241613
Epoch 1 done. train_loss =0.3987785577774048, eval_loss = 0.0021188568063573348, eval_acc=96.04493690366267
Epoch 2 done. train_loss =0.4131927192211151, eval_loss = 0.0029403396517212626, eval_acc=92.61311172668513
roc_auc = 0, f1_score=0, acc=39.900635656424264
Epoch 10, global_loss: 0.017467, auc_roc: 0.000000
Epoch 0 don

...

Epoch 1 done. train_loss =0.29799649119377136, eval_loss = 0.002806057584561725, eval_acc=97.53547423450335
Epoch 2 done. train_loss =0.1881541609764099, eval_loss = 0.001917535640839056, eval_acc=97.53547423450335
Epoch 0 done. train_loss =0.027717970311641693, eval_loss = 9.533239200829222e-05, eval_acc=100.0
Epoch 1 done. train_loss =0.02551320195198059, eval_loss = 5.3243800350772254e-05, eval_acc=100.0
Epoch 2 done. train_loss =0.026885181665420532, eval_loss = 4.426620051164826e-05, eval_acc=100.0
Epoch 0 done. train_loss =4.477056980133057, eval_loss = 0.07710546851158143, eval_acc=0.0
Epoch 1 done. train_loss =4.465577125549316, eval_loss = 0.07341760396957397, eval_acc=0.0
Epoch 2 done. train_loss =4.21795129776001, eval_loss = 0.07089979648590088, eval_acc=0.0
Epoch 0 done. train_loss =0.30481913685798645, eval_loss = 0.001415797621332781, eval_acc=96.86057248384118
Epoch 1 done. train_loss =0.3272854685783386, eval_loss = 0.0013999818618130998, eval_acc=96.73745767928594
Epo

Epoch 0 done. train_loss =0.026095151901245117, eval_loss = 8.956024226091085e-05, eval_acc=100.0
Epoch 1 done. train_loss =0.026664109900593758, eval_loss = 5.1825491733194195e-05, eval_acc=100.0
Epoch 2 done. train_loss =0.02181423455476761, eval_loss = 3.78152172490079e-05, eval_acc=100.0
Epoch 0 done. train_loss =4.7660136222839355, eval_loss = 0.05660836100578308, eval_acc=0.0
Epoch 1 done. train_loss =4.3878703117370605, eval_loss = 0.0590484619140625, eval_acc=0.0
Epoch 2 done. train_loss =4.312482833862305, eval_loss = 0.0538210391998291, eval_acc=0.0
Epoch 0 done. train_loss =0.1672157496213913, eval_loss = 0.0014176913208871962, eval_acc=96.82979378270237
Epoch 1 done. train_loss =0.15753118693828583, eval_loss = 0.0017486518256368546, eval_acc=96.4142813173284
Epoch 2 done. train_loss =0.22682085633277893, eval_loss = 0.0012799273567516718, eval_acc=97.22991689750693
roc_auc = 0, f1_score=0, acc=41.957913578263764
Epoch 82, global_loss: 0.021173, auc_roc: 0.000000
Epoch 0 do

Epoch 2 done. train_loss =0.04191184043884277, eval_loss = 3.819395666145039e-05, eval_acc=100.0
Epoch 0 done. train_loss =4.853479385375977, eval_loss = 0.0702631115913391, eval_acc=0.0
Epoch 1 done. train_loss =4.376651287078857, eval_loss = 0.06459470987319946, eval_acc=0.0
Epoch 2 done. train_loss =4.061707019805908, eval_loss = 0.05836941003799438, eval_acc=6.25
Epoch 0 done. train_loss =0.2966514229774475, eval_loss = 0.0015268641348874323, eval_acc=96.70667897814712
Epoch 1 done. train_loss =0.1669747233390808, eval_loss = 0.001219835748082493, eval_acc=97.38381040320098
Epoch 2 done. train_loss =0.13633736968040466, eval_loss = 0.0014543457007767714, eval_acc=96.84518313327177
roc_auc = 0, f1_score=0, acc=40.86674162975882
Epoch 87, global_loss: 0.021244, auc_roc: 0.000000
Epoch 0 done. train_loss =0.021473538130521774, eval_loss = 4.969787667784789e-05, eval_acc=100.0
Epoch 1 done. train_loss =0.03764040768146515, eval_loss = 3.7812750187624406e-05, eval_acc=100.0
Epoch 2 done

Epoch 0 done. train_loss =0.19813665747642517, eval_loss = 0.0014506214103099933, eval_acc=96.69128962757772
Epoch 1 done. train_loss =0.16616399586200714, eval_loss = 0.0014340558320641628, eval_acc=96.53739612188366
Epoch 2 done. train_loss =0.18902748823165894, eval_loss = 0.0012518952317735386, eval_acc=97.16835949522931
roc_auc = 0, f1_score=0, acc=41.57984716708356
Epoch 92, global_loss: 0.021497, auc_roc: 0.000000
Epoch 0 done. train_loss =0.02160945162177086, eval_loss = 4.310916198906455e-05, eval_acc=100.0
Epoch 1 done. train_loss =0.01826401613652706, eval_loss = 2.894345539735838e-05, eval_acc=100.0
Epoch 2 done. train_loss =0.012897029519081116, eval_loss = 2.453622503152932e-05, eval_acc=100.0
Epoch 0 done. train_loss =0.8283505439758301, eval_loss = 0.007775066680566333, eval_acc=97.16206123973114
Epoch 1 done. train_loss =0.2476777881383896, eval_loss = 0.0025611377812927564, eval_acc=97.23674383868558
Epoch 2 done. train_loss =0.295071005821228, eval_loss = 0.001863870

Epoch 2 done. train_loss =0.1465848982334137, eval_loss = 0.001332538976930552, eval_acc=97.18374884579872
roc_auc = 0, f1_score=0, acc=41.24480804093903
Epoch 97, global_loss: 0.021503, auc_roc: 0.000000
Epoch 0 done. train_loss =0.01666475646197796, eval_loss = 4.282651629294023e-05, eval_acc=100.0
Epoch 1 done. train_loss =0.014869501814246178, eval_loss = 2.6238892910530144e-05, eval_acc=100.0
Epoch 2 done. train_loss =0.015102081000804901, eval_loss = 2.5499348245876585e-05, eval_acc=100.0
Epoch 0 done. train_loss =0.7004474401473999, eval_loss = 0.007378201417018443, eval_acc=97.23674383868558
Epoch 1 done. train_loss =0.48847687244415283, eval_loss = 0.0023394166933591014, eval_acc=97.61015683345781
Epoch 2 done. train_loss =0.20236116647720337, eval_loss = 0.0016809870183601408, eval_acc=97.90888722927558
Epoch 0 done. train_loss =0.019177699461579323, eval_loss = 8.300023879377788e-05, eval_acc=100.0
Epoch 1 done. train_loss =0.02737196907401085, eval_loss = 4.1228257174884055

In [31]:
print(score_base)

[0, 0, 0, 0, 0]
